In [1]:
%load_ext autoreload
%autoreload 2
import torch
import eq
import wandb
from tqdm.notebook import trange
import numpy as np
import warnings
from itertools import product
warnings.filterwarnings('ignore')

In [2]:
device = "cuda"
catalog = eq.catalogs.ANSS_MultiCatalog(mag_completeness=4.5)

Loading existing catalog from /home/zekai/repos/recast/data/ANSS_MultiCatalog.


In [3]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        try:
            dl_train = catalog.train.get_dataloader(batch_size=config.batch_size)
            dl_val = catalog.val.get_dataloader(batch_size=1)
            dl_test = catalog.test.get_dataloader(batch_size=1)

            model = eq.models.RecurrentTPP(context_size=config.context_size,
                                           num_components=config.num_components,
                                           rnn_type=config.rnn_type,
                                           dropout_proba=config.dropout_proba,
                                           learning_rate=config.lr,
                                           tau_mean=config.tau_mean)
            model = model.to(device)

            epochs = 200
            avg_train_loss_list = []
            avg_val_loss_list = []

            optimizer = torch.optim.AdamW(model.parameters(), 
                                          lr=config.lr, 
                                          betas=config.betas, 
                                          weight_decay=config.weight_decay)
            
            best_model_path = "temp_best_model"
            best_val_loss = float('inf')

            for epoch in trange(epochs):
                running_train_loss = []
                model.train()
                for i, data in enumerate(dl_train):
                    data = data.to(device)
                    optimizer.zero_grad()
                    nll = model.nll_loss(data).mean()
                    nll.backward()
                    optimizer.step()
                    running_train_loss.append(nll.item())
                
                model.eval()
                with torch.no_grad():
                    running_val_loss = []
                    for i, data in enumerate(dl_val):
                        data = data.to(device)
                        nll = model.nll_loss(data).mean()
                        running_val_loss.append(nll.item())

                avg_val_loss = np.mean(running_val_loss)
                avg_train_loss = np.mean(running_train_loss)
                avg_train_loss_list.append(avg_train_loss)
                avg_val_loss_list.append(avg_val_loss)
                wandb.log({"avg_train_loss": avg_train_loss,
                           "avg_val_loss": avg_val_loss,
                           "epoch": epoch})


                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    torch.save(model.state_dict(), best_model_path)
                

            best_model = eq.models.RecurrentTPP(context_size=config.context_size,
                                                num_components=config.num_components,
                                                rnn_type=config.rnn_type,
                                                dropout_proba=config.dropout_proba,
                                                learning_rate=config.lr)
            best_model.load_state_dict(torch.load(best_model_path))
            best_model = best_model.to(device)
            best_model.eval()
            with torch.no_grad():
                running_test_loss = []
                for i, data in enumerate(dl_test):
                    data = data.to(device)
                    nll = best_model.nll_loss(data).mean()
                    running_test_loss.append(nll.item())
            avg_test_loss = np.mean(running_test_loss)

        except Exception as e:
            print(e)
            avg_test_loss = float("nan")

        wandb.log({"avg_test_loss": avg_test_loss})


In [4]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "sweep_updated.ipynb"
wandb.login()

wandb: Currently logged in as: zekai-wang. Use `wandb login --relogin` to force relogin


True

In [5]:
sweep_config = {'method': "random"}
metric = {"name": "avg_test_loss",
          "goal": "minimize"}
sweep_config["metric"] = metric

parameter_dict = {"context_size": {'values': [8, 16, 32, 64, 128]},
                  "num_components": {"values": [8, 16, 32, 64, 128]},
                  "rnn_type": {"value": "LSTM"},
                  "dropout_proba": {"values": [0, 0.1, 0.2, 0.3, 0.4, 0.5]},
                  "lr": {"values": [1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2]},
                  "betas": {"values": list(product([0.9, 0.99], [0.99, 0.999, 0.9999]))},
                  "weight_decay": {"values": [0, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2]},
                  "batch_size": {"values": [8, 16, 32, 64, 128]},
                  "tau_mean": {"values": [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1.0]}}

sweep_config["parameters"] = parameter_dict

In [6]:
sweep_id = wandb.sweep(sweep_config, project="RecurrentTPP on ANSS Nov 6")

Create sweep with ID: odgh4jcm
Sweep URL: https://wandb.ai/zekai-wang/RecurrentTPP%20on%20ANSS%20Nov%206/sweeps/odgh4jcm


In [7]:
wandb.agent(sweep_id, function=train, count=200)

wandb: Agent Starting Run: 85ed9gsd with config:
wandb: 	batch_size: 32
wandb: 	betas: [0.99, 0.99]
wandb: 	context_size: 16
wandb: 	dropout_proba: 0.4
wandb: 	lr: 0.05
wandb: 	num_components: 8
wandb: 	rnn_type: LSTM
wandb: 	tau_mean: 0.05
wandb: 	weight_decay: 0.001


  0%|          | 0/200 [00:00<?, ?it/s]

avg_test_loss,▁
avg_train_loss,█▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_val_loss,█▄▄▂▂▂▁▂▂▁▂▁▂▁▁▁▂▁▂▂▁▁▁▂▁▂▁▂▁▂▂▂▂▁▁▂▁▁▁▂
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
avg_test_loss,0.18843
avg_train_loss,-0.22566
avg_val_loss,-2.10463
epoch,199


wandb: Agent Starting Run: ghdivi06 with config:
wandb: 	batch_size: 64
wandb: 	betas: [0.99, 0.999]
wandb: 	context_size: 16
wandb: 	dropout_proba: 0.1
wandb: 	lr: 0.02
wandb: 	num_components: 128
wandb: 	rnn_type: LSTM
wandb: 	tau_mean: 0.2
wandb: 	weight_decay: 0.05


  0%|          | 0/200 [00:00<?, ?it/s]

...

Expected parameter scale (Tensor of shape (8, 2949, 8)) of distribution Weibull(scale: torch.Size([8, 2949, 8]), shape: torch.Size([8, 2949, 8])) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
tensor([[[0.6123, 0.4727, 1.5123,  ..., 0.7554, 1.0979, 0.4845],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         ...,
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan]],

        [[0.6123, 0.4727, 1.5123,  ..., 0.7554, 1.0979, 0.4845],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         ...,
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,  

avg_train_loss,█▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂
avg_val_loss,▆▃▃▃▁▃▂▁▃▂▁▂▂▂▁▁▃▁▁▁▁▂▁▁▁▃▁▂▁▂▁▂▂▁▂▂▃▁▁█
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
avg_test_loss,nan
avg_train_loss,-0.21221
avg_val_loss,-1.433
epoch,43


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: e5a5v549 with config:
wandb: 	batch_size: 64
wandb: 	betas: [0.9, 0.99]
wandb: 	context_size: 8
wandb: 	dropout_proba: 0.2
wandb: 	lr: 0.005
wandb: 	num_components: 64
wandb: 	rnn_type: LSTM
wandb: 	tau_mean: 0.02
wandb: 	weight_decay: 0.02


  0%|          | 0/200 [00:00<?, ?it/s]

avg_test_loss,▁
avg_train_loss,█▄▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_val_loss,█▄▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
avg_test_loss,0.17205
avg_train_loss,-0.16545
avg_val_loss,-2.1123
epoch,199
